In [2]:
USE Chinook;

Commands completed successfully.

Total execution time: 00:00:00

In [3]:
-- Obtener el dinero recaudado por cada empleado durante cada a˜no.

SELECT e.EmployeeId, YEAR(i.InvoiceDate) as 'Year', SUM(i.Total) as 'Total'
FROM Employee e
INNER JOIN Customer c ON c.SupportRepId = e.EmployeeId
INNER JOIN Invoice i ON i.CustomerId = c.CustomerId
GROUP BY e.EmployeeId, YEAR(i.InvoiceDate)

(15 rows affected)

Total execution time: 00:00:00.022

EmployeeId,Year,Total
3,2009,123.75
4,2009,161.37
5,2009,164.34
3,2010,221.92
4,2010,122.76
5,2010,136.77
3,2011,184.34
4,2011,125.77
5,2011,159.47
3,2012,146.60


In [4]:
-- Obtener la cantidad de pistas de audio que tengan una duraci´on superior a la duraci´on promedio de todas las pistas de audio. Adem´as, obtener la
-- sumatoria de la duraci´on de todas esas pistas en minutos.

SELECT COUNT(*) as 'Cantidad', SUM(t.Milliseconds/60000) as 'Sumatoria'
FROM Track t 
WHERE t.Milliseconds > (select AVG(t2.Milliseconds) from Track t2)

-- Nooooooooooooo esto no esta bien, no son todos los tracks de audio, hay algunos que no son audio, da un resultado INCORRECTO

-- ESTO QUE SIGUE ESTA BIEN --> FILTRAMOS POR TRACKS DE AUDIO

SELECT COUNT(*) as 'Cantidad', SUM(t.Milliseconds/60000) as 'Sumatoria'
FROM Track t
INNER JOIN MediaType m ON m.MediaTypeId = t.MediaTypeId
WHERE m.Name LIKE '%audio%' 
    AND t.Milliseconds > (
        SELECT AVG(t2.Milliseconds)
        FROM Track t2
        INNER JOIN MediaType m2 ON m2.MediaTypeId = t2.MediaTypeId
        WHERE m2.Name LIKE '%audio%'
    )

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.048

Cantidad,Sumatoria
494,10542


Cantidad,Sumatoria
1345,7260


In [5]:
-- Crear una vista que devuelva las playlists que tienen al menos una pista del g´enero “Rock”.

CREATE OR ALTER VIEW playlists_rock AS
SELECT p.PlaylistId, p.Name
FROM Playlist p
WHERE p.PlaylistId in (
    SELECT p2.PlaylistId
    FROM Playlist p2
    INNER JOIN PlaylistTrack pt on pt.PlaylistId = p2.PlaylistId
    INNER JOIN Track t on t.TrackId = pt.TrackId
    INNER JOIN Genre g on g.GenreId = t.GenreId
    WHERE g.Name = 'Rock'
)

Commands completed successfully.

Total execution time: 00:00:00.012

In [6]:
-- Utilizando la vista, obtener la cantidad de playlists que no poseen pistas de dicho g´enero.
SELECT COUNT(*) AS 'Cantidad'
FROM Playlist p
WHERE p.PlaylistId NOT IN (SELECT PlaylistId FROM playlists_rock)

(1 row affected)

Total execution time: 00:00:00.033

Cantidad
13


In [7]:
-- Obtener las playlists m´as caras. Hint: primero obtener el ‘precio’ de cada playlist.

WITH precios AS (
    SELECT pt.PlaylistId, SUM(t.UnitPrice) as 'precio'
    FROM PlaylistTrack pt
    INNER JOIN Track t ON t.TrackId = pt.TrackId
    GROUP BY pt.PlaylistId
)

SELECT p.PlaylistId, p.Name 
FROM Playlist p
INNER JOIN precios pr on pr.PlaylistId = p.PlaylistId
WHERE pr.precio in (SELECT MAX(pr2.precio) FROM precios pr2)


(2 rows affected)

Total execution time: 00:00:00.023

PlaylistId,Name
1,Music
8,Music
